In [41]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [42]:
import os
from pathlib import Path
os.chdir(Path("C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops\\"))

In [43]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [44]:
# from software_defect_prediction.constants import *
from software_defect_prediction.utils.common import *
# from software_defect_prediction.entity.config_entity import DataIngestionConfig
# from software_defect_prediction.config.configuration import ConfigurationManager

In [45]:
from dataclasses import dataclass
from pathlib import Path
from software_defect_prediction import logger

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    source_file_path : str
    input_file_name : str
    STATUS_FILE: Path 

In [46]:
from venv import create
from software_defect_prediction.constants import *
from software_defect_prediction.entity.config_entity import DataIngestionConfig
from software_defect_prediction.utils.common import read_yaml, create_directories
from box import ConfigBox
# from software_defect_prediction.entity.config_entity import (DataIngestionConfig)

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            source_URL_file_name = config.source_URL_file_name,
            source_URL_unzip_file_name = config.source_URL_unzip_file_name,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        
        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            input_file_name = config.input_file_name,
            source_file_path = config.source_file_path,
            STATUS_FILE = config.STATUS_FILE
        )
        return data_validation_config
    
    def get_data_schema(self) -> ConfigBox:
        return(self.schema)
    

In [48]:
from asyncio.windows_events import NULL
from msilib import schema
from pickle import TRUE
from software_defect_prediction import logger

from pathlib import Path
import shutil
import pandas as pd

class Data_Validation():
    def __init__(self,data_validation_config : DataValidationConfig, data_schema : ConfigBox) -> None:
        self.config = data_validation_config
        self.data_schema = data_schema
        
    def prepare_files(self) -> None:
        
        destination_file_path = Path(Path(self.config.root_dir) / Path(self.config.input_file_name))
        if os.path.exists(destination_file_path):
            os.remove(destination_file_path)
        
        shutil.copy(self.config.source_file_path,self.config.root_dir)

    def validate_columns(self) :
        input_data = pd.read_csv(Path(self.config.root_dir) / Path(self.config.input_file_name))
        
        input_data_schema = pd.DataFrame({
            "column" : input_data.columns,
            "data_type" : input_data.dtypes
        }).reset_index(drop=TRUE)
                
        original_data_schema = pd.DataFrame({
            "column" : self.data_schema.COLUMNS.keys(),
            "data_type" : self.data_schema.COLUMNS.values()
            
        }).reset_index(drop=TRUE)
        
        # Merge the two DataFrames on both column names and data types
        merged_df = input_data_schema.astype(str).merge(original_data_schema.astype(str), on=['column', 'data_type'], how='outer', indicator=True)

        # Filter rows where the merge indicator is not 'both' (i.e., where there is a mismatch)
        mismatched_columns = merged_df[merged_df['_merge'] != 'both']

        # Print columns causing the mismatch
        if not mismatched_columns.empty:
            print("Columns causing the mismatch:")
            print(mismatched_columns['column'])
        else:
            print("Columns and data types match between the two DataFrames.")
        
        return merged_df

data_validation_obj = Data_Validation(ConfigurationManager().get_data_validation_config(),ConfigurationManager().get_data_schema())
data_validation_obj.prepare_files()
input_df = data_validation_obj.validate_columns()

input_df
# list(data_validation_obj.data_schema.COLUMNS.keys())
# list(data_validation_obj.data_schema.COLUMNS.values())


2024-05-26 13:35:37.724 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: config\config.yaml loaded successfully
2024-05-26 13:35:37.726 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: params.yaml loaded successfully
2024-05-26 13:35:37.730 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: schema.yaml loaded successfully
2024-05-26 13:35:37.732 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts
2024-05-26 13:35:37.734 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts/data_validation
2024-05-26 13:35:37.737 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: config\config.yaml loaded successfully
2024-05-26 13:35:37.739 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: params.yaml loaded successfully
2024-05-26 13:35:37.744 | INF

Columns and data types match between the two DataFrames.


,column,data_type,_merge
0,b,float64,both
1,branchCount,float64,both
2,d,float64,both
3,defects,bool,both
4,e,float64,both
5,ev(g),float64,both
6,i,float64,both
7,id,int64,both
8,iv(g),float64,both
9,l,float64,both


In [22]:
# configuration_obj = ConfigurationManager()
# data_ingestion_conf =  configuration_obj.get_data_ingestion_config()
# step_data_ing = Data_Ingestion(data_ingestion_conf)
# step_data_ing.download_file()
# step_data_ing.unzip_file()
# # logger.success("Data ingestion completed successfully")

In [23]:
# data_ingestion_conf.root_dir